In [ ]:
df_with_components = calculate_partition_components(partition_folder,60)

df_exploded = df_with_components.select('file',  F.explode('components').alias('component'))
df_split = df_exploded.select("file", col("component.seqID").alias("seqID"), col("component.component_id").alias("component_id"))

# # Ajouter une colonne d'identifiant unique basée sur le couple (file, component_id)
windowSpec = Window.orderBy("file", "component_id")
df = df_split.withColumn("new_component_id", dense_rank().over(windowSpec))
new_df = df.select("seqID","new_component_id")

# # new_df = df_split.select(
# #             'seqID',
# #             concat(col('file'), lit(':'), col('component_id')).alias('new_component_id')
# #         )

# #cration d'un graph ou chaque noeud est le numero d'une composante (new_graph)

components_per_node = new_df \
    .groupby("seqID") \
    .agg(collect_set("new_component_id").alias("components")) \
    .withColumn("min_component", array_min("components"))

# # components_per_node = new_df \
# #     .groupby("seqID") \
# #     .agg(collect_set("new_component_id").alias("components")) \
# #     .withColumn("min_component", array_min("components"))


components_per_node_exploded = components_per_node.select('seqID', 'min_component', F.explode('components').alias('component'))
result = components_per_node_exploded.where(col("component") > col("min_component")).drop("seqID").dropDuplicates()


new_graph = result.withColumnRenamed("min_component", "src").withColumnRenamed("component", "dst")


new_graph.select('src', 'dst').write.csv('/data/bd/dataset/proteine/80_80/G99/graph_partitionned/aretes_g_prime.csv')

# # #calcul des composantes sur le nouveau graph
# edges =new_graph.select('src', 'dst').collect()
# composantes_df = components_2(edges)

# composantes_df_exploded = composantes_df \
#         .groupby("component") \
#         .agg(collect_set("partial_component").alias("partial_components")) \
#         .withColumn("min_partial_component", array_min("partial_components")) \
#         .selectExpr("explode(partial_components) as partial_component" , "min_partial_component")

# # #resultat final en faisant une jointure gauche
# result = components_per_node.join(composantes_df_exploded, components_per_node.min_component == composantes_df_exploded.partial_component, "left_outer") \
#            .select(components_per_node.seqID, when(composantes_df_exploded.partial_component.isNull(), components_per_node.min_component).otherwise(composantes_df_exploded.min_partial_component).alias("new_component_id"))


# #ecriture du résultat dans des fichier parquet
# file_path = f"/data/bd/dataset/proteine/80_80/G99/graph_nodeID_only"
# result.repartition(1).write.mode("overwrite").parquet(file_path)






#--------------------
edges = spark.read.csv('/data/bd/dataset/proteine/80_80/G99/graph_partitionned/aretes_g_prime.csv', header=False, inferSchema=True)

# Création du graph avec igraph
from igraph import Graph

# Conversion des arêtes DataFrame en liste de tuples
edges_list = edges.rdd.map(lambda row: (row[0], row[1])).collect()

# Création du graph igraph
ig_graph = Graph.TupleList(edges_list, directed=True)


#___________
composantes_df = components_2(edges_list)
